In [20]:
%%html
<style type='text/css'>
.CodeMirror{
    font-size: 28px;
}

div.output_area pre {
    font-size: 20px;
}
</style>

In [21]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error

from plotly.offline import plot
import plotly.graph_objs as go

from utils import prepare_data

join = os.path.join

In [22]:
base_url = "exp_257"
sample_data_csv = pd.read_csv(join(base_url, "val_result.csv"))

In [23]:
display(sample_data_csv.head(5))

,subject_id,seg_id,slices,DSC,Diff_Dice
0,BraTS2021_01201,1927,106_107_108_109_129,0.3907619118690491/0.3384034335613251,0.915888
1,BraTS2021_01201,1927,106_107_108_109_129,0.3907619118690491/0.3384034335613251,0.915888
2,BraTS2021_01201,1927,106_107_108_109_129,0.3907619118690491/0.3384034335613251,0.915888
3,BraTS2021_01201,1927,106_107_108_109_129,0.3907619118690491/0.3384034335613251,0.915888
4,BraTS2021_01201,1927,106_107_108_109_129,0.3907619118690491/0.3384034335613251,0.915888


In [24]:
(dice_pred, dice_true), urls = prepare_data(sample_data_csv, base_url)

# url Parse
### redict page: template.html
### base_url: your base dir to contain your images
### sid: subject id
### id: image id
### slice: slices to visualize 
### DSC: predicted / true Dice
### Diff Dice: error map dice

In [25]:
print(urls[0])

./template.html?base_url=exp_257&sid=BraTS2021_01201&id=1927&slice=106_107_108_109_129&mode=val&Diff_Dice=0.916&DSC=0.391 / 0.338


In [26]:
mae = mean_absolute_error(dice_pred, dice_true)
r2 = r2_score(dice_pred, dice_true)

In [27]:
# one-line scatter plot of dice
data = [
        go.Scatter(
                x=dice_pred,
                y=dice_true,
                mode='markers',
                marker=dict(size=14),
                name=f"DSC [MAE={mae:.3f} R^2={r2:.3f}]",
                text=[],
                customdata=urls) # pass your custom urls
       ]

# Plot the diagonal line
xx = np.linspace(0, 1, 1000)
data.append(
        go.Scatter(x=xx, y=xx, mode='lines', 
                       line=dict(color='firebrick', width=8),
                       name='Diagonal')
        )

In [28]:
# Build layout
layout = go.Layout(
        hovermode='closest',
        xaxis_title="Dice Prediction",
        yaxis_title="Dice Ground-Truth",
    )

# Build Figure
fig = go.Figure(
        data=data,
        layout=layout,
    )

In [29]:
# Get HTML representation of plotly.js and this figure
# Convert your plot to a html <div> element
plot_div = plot(fig, output_type='div', include_plotlyjs=True)

# Get id of html div element that looks like
# <div id="301d22ab-bfba-4621-8f5d-dc4fd855bb33" ... >
res = re.search('<div id="([^"]*)"', plot_div)
div_id = res.groups()[0]
print(f"<div id={div_id}>")

# Build JavaScript callback for handling clicks
# and opening the URL in the trace's customdata 
js_callback = """
    <script>
    var plot_element = document.getElementById("{div_id}");
    plot_element.on('plotly_click', function(data){{
        console.log(data);
        var point = data.points[0];
        if (point) {{
            console.log(point.customdata);
            window.open(point.customdata);
        }}
    }})
    </script>
    """.format(div_id=div_id)
print(js_callback)

# Build HTML string
html_str = """
    <html>
    <body>
    {plot_div} # your plot_div element
    {js_callback} # your javascript callback function
    </body>
    </html>
    """.format(plot_div=plot_div, js_callback=js_callback)

<div id=93d7cc97-2c03-4c7e-a8ec-ca2a2b1b9448>

    <script>
    var plot_element = document.getElementById("93d7cc97-2c03-4c7e-a8ec-ca2a2b1b9448");
    plot_element.on('plotly_click', function(data){
        console.log(data);
        var point = data.points[0];
        if (point) {
            console.log(point.customdata);
            window.open(point.customdata);
        }
    })
    </script>
    


In [30]:
# Write out HTML file
with open(f'demo_custom_html.html', 'w') as f:
    f.write(html_str)